In [1]:
search_for = 139
start_from = 0
threads = 10
thread = 3
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.08947014808654785
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 376006121
type: [1, 1, 1, 1, 139] 139
cases of this type: 373301041
100000 10.838167788344617
200000 19.939170639293835
300000 20.06189692733251
400000 22.09706989924384
500000 20.319451605519856
600000 21.663648704929884
700000 22.75355054989471
800000 21.74820796953383
900000 21.388245442078198
1000000 20.45264298771537
1100000 21.63494704194311
1200000 21.919418977797065
1300000 20.75429048064928
1400000 3.8184317306369957
1500000 21.010470601176007
1600000 18.771090113888768
1700000 22.354642470114403
1800000 20.87525859058906
1900000 22.701519547331863
2000000 18.538886043119177
2100000 25.663817775131445
2200000 22.406893384514404
2300000 26.804423323192825
2400000 30.86344150989943
2500000 39.84987471053934
2600000 36.71174669744218
2700000 21.588577511685934
2800000 27.679450316129884
2900000 47.74907926970359
3000000 44.8

29700000 37.5077758863355
29800000 14.10280093240113
29900000 35.55960268063622
30000000 37.32822706116349
30100000 37.936097211157865
30200000 36.566029840935414
30300000 37.982025203216246
30400000 36.25639011961711
30500000 37.909184085620424
30600000 35.20809151908556
30700000 38.45853397229721
30800000 32.728796539539395
30900000 16.927421401632834
31000000 27.04340239245328
31100000 35.50873550995269
31200000 34.29524976855086
31300000 38.95562860431696
31400000 35.53110455569943
31500000 36.87127141351977
31600000 35.37367442070784
31700000 39.15249336933751
31800000 39.33092492792468
31900000 39.881444528634304
32000000 25.689396055614203
32100000 24.21794088434879
32200000 37.39078670975599
32300000 38.17929889932214
32400000 37.42010288965386
32500000 15.171647377438699
32600000 33.809749638152866
32700000 37.13144041395412
32800000 37.24261320560996
32900000 36.926094022220624
33000000 36.37081263013192
33100000 34.937782551537126
33200000 37.43868256438656
33300000 35.06248

59700000 33.62160773438859
59800000 33.739083744114666
59900000 34.26648837501266
60000000 30.173013762843553
60100000 34.61582601587947
60200000 29.704860248027998
60300000 32.79206645696046
60400000 25.519080993732786
60500000 13.797795193425403
60600000 30.020110681675686
60700000 33.04920813073681
60800000 32.318881839336115
60900000 31.581800921292245
61000000 34.75546696367759
61100000 32.38945767435839
61200000 34.92895765974318
61300000 33.29840350248797
61400000 14.649482617995707
61500000 36.61141732353991
61600000 33.96674394022982
61700000 35.648940270984156
61800000 35.15859778194156
61900000 33.17799976889696
62000000 33.184576998127106
62100000 35.15153147237257
62200000 27.527409637166784
62300000 16.984657984239945
62400000 32.27922135506807
62500000 32.606229132151505
62600000 31.239217051065467
62700000 33.84553565571919
62800000 32.18016020063568
62900000 28.576218908073752
63000000 30.938298492767554
63100000 20.56357248009206
63200000 17.11966296182559
63300000 30

89500000 27.434717392925894
89600000 27.429666389436886
89700000 27.47310766043553
89800000 27.175292166476122
89900000 27.356738082886775
90000000 9.267407336447366
90100000 25.91464758949711
90200000 27.319487273022826
90300000 27.38127078603508
90400000 26.281754728648416
90500000 26.673747542643955
90600000 28.11018805748127
90700000 11.144768084153167
90800000 28.67358648627736
90900000 29.096820675341874
91000000 30.817229612091932
91100000 30.078353112953767
91200000 30.363851895137326
91300000 31.514144571485883
91400000 33.16326240419866
91500000 30.499860357880998
91600000 29.659288118681413
91700000 30.421801439441513
91800000 28.611652628648255
91900000 29.410155361295892
92000000 12.918355230191523
92100000 22.557195503134956
92200000 27.5418463352613
92300000 27.660764259810826
92400000 26.081221556471526
92500000 27.134283363688123
92600000 27.710562680409517
92700000 4.735151499444426
92800000 29.12581525654821
92900000 27.53263442852009
93000000 26.149395175583887
9310

118500000 25.68083515473246
118600000 26.74062131477142
118700000 27.207413967829584
118800000 23.414137264610737
118900000 25.280571197288577
119000000 19.614475626867147
119100000 13.109150835375365
119200000 22.91839494785586
119300000 24.735600318549732
119400000 23.099588593704336
119500000 18.42077863447059
119600000 13.552522697624855
119700000 24.42918581479917
119800000 23.301599446138045
119900000 24.87198257298359
120000000 14.538168205674591
120100000 18.1371970886304
120200000 24.602093817728832
120300000 24.67359191755373
120400000 23.97665764796898
120500000 25.818776783578805
120600000 25.243773903016287
120700000 27.706846543390046
120800000 26.880289531655443
120900000 24.945231545252
121000000 26.8348290404974
121100000 25.639335219173613
121200000 26.086090528527244
121300000 26.33679777418377
121400000 25.63266061236278
121500000 24.605182045515463
121600000 22.75620775479529
121700000 22.613300243013807
121800000 9.640927325794342
121900000 24.313053654716327
1220

...

233100000 13.08145711951856
233200000 13.251560807185937
233300000 12.879968386581222
233400000 14.205267927884037
233500000 13.715334202832983
233600000 14.345695586303869
233700000 13.491019126557058
233800000 14.414114115851556
233900000 13.531061779636634
234000000 14.201188701775388
234100000 12.740977310854841
234200000 13.12894907933546
234300000 11.952044311388757
234400000 12.676248257190371
234500000 12.499566106544973
234600000 12.754548403941001
234700000 4.765636350297569
234800000 12.76932321402334
234900000 12.037008124876385
235000000 6.883947714757231
235100000 9.180477884096415
235200000 12.645031734116701
235300000 11.769345235091134
235400000 4.3267247420556085
235500000 12.864300041876103
235600000 12.131440010150518
235700000 12.901345003721932
235800000 12.560407916080132
235900000 13.172950700404161
236000000 12.434406405118915
236100000 14.379089740280644
236200000 13.01606709655036
236300000 14.201197649421799
236400000 12.725606125760043
236500000 13.86887355

261700000 9.74244402128921
261800000 11.260926855766815
261900000 1.8636504462008552
262000000 11.366594630001995
262100000 9.748567947649653
262200000 11.224745177704827
262300000 9.725227327523719
262400000 6.440035707461958
262500000 7.925474718725297
262600000 10.645934516088218
262700000 10.43980714310065
262800000 11.343928018686436
262900000 11.688419418703381
263000000 11.102372744152664
263100000 11.110100572724974
263200000 11.497652591701977
263300000 11.232322309427797
263400000 11.245549072769219
263500000 11.261017401913719
263600000 10.936082034236748
263700000 10.947641328936536
263800000 10.445713866423711
263900000 10.340923087070284
264000000 5.284160116405145
264100000 9.429925844146307
264200000 9.697773457164038
264300000 10.69940392479586
264400000 9.38664724567973
264500000 9.703397106023722
264600000 3.3960127236771176
264700000 10.95026397730845
264800000 9.39422722185755
264900000 10.501874391501866
265000000 9.354441169483577
265100000 6.0909211055095165
265

290800000 8.624621785280304
290900000 7.658976004266243
291000000 7.886888959149151
291100000 7.378444855154141
291200000 8.065389057213448
291300000 7.490094811169317
291400000 4.298496832589083
291500000 6.139073527857155
291600000 7.791507239345844
291700000 7.587356499258762
291800000 7.5247899169941945
291900000 7.588092031232752
292000000 7.889550871855303
292100000 7.974277047147394
292200000 2.508341470641629
292300000 8.700843305712333
292400000 8.349593274828369
292500000 8.864544060621746
292600000 8.156528516729018
292700000 9.282427822603797
292800000 8.0067724037498
292900000 9.333988502171701
293000000 7.8100591837972155
293100000 9.053392456602388
293200000 7.761742649653094
293300000 5.747355618675382
293400000 5.679831471014752
293500000 8.216764715606386
293600000 7.303241593869272
293700000 7.933293303605737
293800000 6.985840153394727
293900000 7.852143967311299
294000000 7.089487430845659
294100000 3.002561473918962
294200000 6.766948981248003
294300000 7.70891003

320100000 5.601589689961748
320200000 5.843698465843814
320300000 5.670403328451585
320400000 5.425105425534476
320500000 5.505271969175804
320600000 5.302826678364384
320700000 5.201191879696224
320800000 4.903410436364005
320900000 4.562643113574052
321000000 1.8836900935746888
321100000 5.3714826274399865
321200000 4.8619961506984595
321300000 5.532680192676605
321400000 5.14425562745794
321500000 6.059641835731135
321600000 5.071004676222323
321700000 6.024730239494354
321800000 5.298923181371486
321900000 3.7463131849162634
322000000 3.972913130868509
322100000 5.994063957113761
322200000 5.438648042558878
322300000 5.978025654979571
322400000 5.611114116440584
322500000 5.901285487302764
322600000 4.976229352820134
322700000 2.4753886918852808
322800000 5.713830903525959
322900000 5.9301385352550495
323000000 5.81632230331325
323100000 5.489935542432354
323200000 5.511846655687091
323300000 5.38747881695861
323400000 5.2892609243797795
323500000 5.0767915816840095
323600000 3.957

349100000 3.1039107461173834
349200000 2.6770811895797078
349300000 1.8053822034158442
349400000 2.000728109140897
349500000 2.9182913703700293
349600000 2.629164191068657
349700000 2.946818708166639
349800000 2.7636016318141774
349900000 2.7772588247157257
350000000 2.5650516033774315
350100000 2.855831754588886
350200000 2.6255811646350105
350300000 2.765027567780161
350400000 2.449779823422242
350500000 0.94966633729649
350600000 2.1877448831455446
350700000 2.7711588398441296
350800000 2.576734840034064
350900000 2.6933980469941434
351000000 2.53920600120982
351100000 2.6456657461560895
351200000 2.594950543573306
351300000 2.662521308163316
351400000 2.582445744978084
351500000 2.6268394070096175
351600000 2.7978004320350673
351700000 0.5951209488790141
351800000 2.810491504218078
351900000 2.5196432500478645
352000000 0.9271554182191305
352100000 2.445323248144831
352200000 2.6466671019553334
352300000 2.394944783139919
352400000 2.5755252475538235
352500000 2.4437513580529147
35